In [ ]:
from IPython import get_ipython
from typing_extensions import Annotated
from dotenv import load_dotenv, find_dotenv

import autogen
from autogen.cache import Cache
import os

load_dotenv(find_dotenv())

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4o", "gpt4-turbo"],
    },
)

settings = {
    'host': os.getenv('ACCOUNT_HOST'),
    'master_key': os.getenv('ACCOUNT_KEY'),
    'database_id': os.getenv('COSMOS_DATABASE'),
    'container_id': os.getenv('COSMOS_CONTAINER'),
}

for key, value in settings.items():
    if value is None:
        raise ValueError(f"Missing environment variable for {key}")

In [ ]:
onboarding_buddy_prompt = """
You are an experienced onboarding buddy, working with a new hire. You need to guide the user through the onboarding process. You have all tthe information you need to help the user onboard. You are responsible for ensuring the user is onboarded successfully.
If you do not know the answer to a question, you can ask the supervisor for help.    
"""

supervisor_prompt = """
You are a supervisor. You are responsible for ensuring that the onboarding buddy is able to guide the new hire through the onboarding process. You need to provide the onboarding buddy with the information they need to help the new hire onboard successfuly.
You are responsible for the onboarding process and ensuring that the new hire is onboarded successfully.
"""

In [ ]:
from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

llm_config = {"config_list": config_list, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="The new hire.",
    human_input_mode="ALWAYS",
    code_execution_config=False
)

onboarding_buddy = autogen.ConversableAgent(
    name="onboarding_buddy",
    system_message= onboarding_buddy_prompt,
    llm_config=llm_config,
)
supervisor = autogen.ConversableAgent(
    name="supervisor",
    system_message=supervisor_prompt,
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(agents=[user_proxy, onboarding_buddy, supervisor], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)